In [1]:
import os
import csv
import glob
import h5py
import random
import scipy.ndimage
import numpy as np
import tensorflow as tf
from imutils import paths
from keras.applications.inception_v3 import InceptionV3
from sklearn.preprocessing import LabelEncoder
from keras.applications import imagenet_utils
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img

Using TensorFlow backend.


In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [3]:
ROOT_DIR = os.path.normpath(os.path.join(os.path.dirname(os.path.realpath('__file__'))))
SOURCE_DIR = os.path.abspath('/data/aumkar/train')
TARGET_DIR = os.path.abspath('/data/aumkar/validation')
DATA_LOAD_DIR = os.path.abspath('/data/aumkar/data_load')

In [4]:
source_path = list(paths.list_images(SOURCE_DIR))
target_path = list(paths.list_images(TARGET_DIR))

In [ ]:
DIR_LIST1 = ['aeroplane', 'bicycle', 'horse']
DIR_LIST2 = ['knife', 'person', 'plant']
DIR_LIST3 = ['train', 'truck', 'bus']
DIR_LIST4 = ['car', 'motorcycle', 'skateboard']

In [ ]:
dir_list1 = [DIR_LIST1, DIR_LIST2]
dir_list2 = [DIR_LIST3, DIR_LIST4]

In [5]:
random.shuffle(source_path)

In [6]:
random.shuffle(target_path)

In [7]:
labels_source = [p.split(os.path.sep)[-2] for p in source_path]
labels_target = [p.split(os.path.sep)[-2] for p in target_path]

In [8]:
le = LabelEncoder()
labels = le.fit_transform(labels_source)
labels_t = le.fit_transform(labels_target)

In [9]:
cols_inception = ["feat_{}".format(i) for i in range(0, 5 * 5 * 2048)]
cols_inception = ["class"] + cols_inception

csv_inception = open(os.path.join(DATA_LOAD_DIR, 'inception.csv'), 'w')
csv_inception.write('{}\n'.format(','.join(cols_inception)))

In [15]:
cols_inception_target = ["feat_{}".format(i) for i in range(0, 5 * 5 * 2048)]
cols_inception_target = ["class"] + cols_inception_target

csv_inception_target = open(os.path.join(DATA_LOAD_DIR, 'inception_target.csv'), 'w')
csv_inception_target.write('{}\n'.format(','.join(cols_inception_target)))

In [11]:
with tf.device('/gpu:1'):
    model_ = InceptionV3(weights = 'imagenet', include_top = False)

In [16]:
def feature_extract(path, batch_size, labels):
    
    for (b, i) in enumerate(range(0, len(path), batch_size)):
        # extract the batch of images and labels, then initialize the
        # list of actual images that will be passed through the network
        # for feature extraction
        print("[INFO] processing batch {}/{}".format(b + 1, int(np.ceil(len(path) / float(batch_size)))))
        batchPaths = path[i: i + batch_size]
        batchLabels = labels[i: i + batch_size]
        batchImages = []

        # loop over the images and labels in the current batch
        for imagePath in batchPaths:
            # load the input image using the Keras helper utility while
            # ensuring the image is resized to 224x224 pixels
            image = load_img(imagePath, target_size=(224, 224))
            image = img_to_array(image)

            # preprocess the image by (1) expanding the dimensions and
            # (2) subtracting the mean RGB pixel intensity from the
            # ImageNet dataset
            image = np.expand_dims(image, axis=0)
            image = imagenet_utils.preprocess_input(image)

            # add the image to the batch
            batchImages.append(image)

        # pass the images through the network and use the outputs as our
        # actual features, then reshape the features into a flattened
        # volume
        batchImages = np.vstack(batchImages)
        features = model_.predict(batchImages, batch_size = batch_size)
        features = features.reshape((features.shape[0], 5 * 5 * 2048))

        # loop over the class labels and extracted features
        for (label, vec) in zip(batchLabels, features):
            # construct a row that exists of the class label and extracted features
            vec = ",".join([str(v) for v in vec])
            csv_inception_target.write("{},{}\n".format(label, vec))

    # close the CSV file
    csv_inception_target.close()

In [13]:
feature_extract(source_path, 64, labels)

[INFO] processing batch 1/2382
[INFO] processing batch 2/2382
[INFO] processing batch 3/2382
[INFO] processing batch 4/2382
[INFO] processing batch 5/2382
[INFO] processing batch 6/2382
[INFO] processing batch 7/2382
[INFO] processing batch 8/2382
[INFO] processing batch 9/2382
[INFO] processing batch 10/2382
[INFO] processing batch 11/2382
[INFO] processing batch 12/2382
[INFO] processing batch 13/2382
[INFO] processing batch 14/2382
[INFO] processing batch 15/2382
[INFO] processing batch 16/2382
[INFO] processing batch 17/2382
[INFO] processing batch 18/2382
[INFO] processing batch 19/2382
[INFO] processing batch 20/2382
[INFO] processing batch 21/2382
[INFO] processing batch 22/2382
[INFO] processing batch 23/2382
[INFO] processing batch 24/2382
[INFO] processing batch 25/2382
[INFO] processing batch 26/2382
[INFO] processing batch 27/2382
[INFO] processing batch 28/2382
[INFO] processing batch 29/2382
[INFO] processing batch 30/2382
[INFO] processing batch 31/2382
[INFO] processing

[INFO] processing batch 253/2382
[INFO] processing batch 254/2382
[INFO] processing batch 255/2382
[INFO] processing batch 256/2382
[INFO] processing batch 257/2382
[INFO] processing batch 258/2382
[INFO] processing batch 259/2382
[INFO] processing batch 260/2382
[INFO] processing batch 261/2382
[INFO] processing batch 262/2382
[INFO] processing batch 263/2382
[INFO] processing batch 264/2382
[INFO] processing batch 265/2382
[INFO] processing batch 266/2382
[INFO] processing batch 267/2382
[INFO] processing batch 268/2382
[INFO] processing batch 269/2382
[INFO] processing batch 270/2382
[INFO] processing batch 271/2382
[INFO] processing batch 272/2382
[INFO] processing batch 273/2382
[INFO] processing batch 274/2382
[INFO] processing batch 275/2382
[INFO] processing batch 276/2382
[INFO] processing batch 277/2382
[INFO] processing batch 278/2382
[INFO] processing batch 279/2382
[INFO] processing batch 280/2382
[INFO] processing batch 281/2382
[INFO] processing batch 282/2382
[INFO] pro

[INFO] processing batch 502/2382
[INFO] processing batch 503/2382
[INFO] processing batch 504/2382
[INFO] processing batch 505/2382
[INFO] processing batch 506/2382
[INFO] processing batch 507/2382
[INFO] processing batch 508/2382
[INFO] processing batch 509/2382
[INFO] processing batch 510/2382
[INFO] processing batch 511/2382
[INFO] processing batch 512/2382
[INFO] processing batch 513/2382
[INFO] processing batch 514/2382
[INFO] processing batch 515/2382
[INFO] processing batch 516/2382
[INFO] processing batch 517/2382
[INFO] processing batch 518/2382
[INFO] processing batch 519/2382
[INFO] processing batch 520/2382
[INFO] processing batch 521/2382
[INFO] processing batch 522/2382
[INFO] processing batch 523/2382
[INFO] processing batch 524/2382
[INFO] processing batch 525/2382
[INFO] processing batch 526/2382
[INFO] processing batch 527/2382
[INFO] processing batch 528/2382
[INFO] processing batch 529/2382
[INFO] processing batch 530/2382
[INFO] processing batch 531/2382
[INFO] pro

[INFO] processing batch 751/2382
[INFO] processing batch 752/2382
[INFO] processing batch 753/2382
[INFO] processing batch 754/2382
[INFO] processing batch 755/2382
[INFO] processing batch 756/2382
[INFO] processing batch 757/2382
[INFO] processing batch 758/2382
[INFO] processing batch 759/2382
[INFO] processing batch 760/2382
[INFO] processing batch 761/2382
[INFO] processing batch 762/2382
[INFO] processing batch 763/2382
[INFO] processing batch 764/2382
[INFO] processing batch 765/2382
[INFO] processing batch 766/2382
[INFO] processing batch 767/2382
[INFO] processing batch 768/2382
[INFO] processing batch 769/2382
[INFO] processing batch 770/2382
[INFO] processing batch 771/2382
[INFO] processing batch 772/2382
[INFO] processing batch 773/2382
[INFO] processing batch 774/2382
[INFO] processing batch 775/2382
[INFO] processing batch 776/2382
[INFO] processing batch 777/2382
[INFO] processing batch 778/2382
[INFO] processing batch 779/2382
[INFO] processing batch 780/2382
[INFO] pro

[INFO] processing batch 1000/2382
[INFO] processing batch 1001/2382
[INFO] processing batch 1002/2382
[INFO] processing batch 1003/2382
[INFO] processing batch 1004/2382
[INFO] processing batch 1005/2382
[INFO] processing batch 1006/2382
[INFO] processing batch 1007/2382
[INFO] processing batch 1008/2382
[INFO] processing batch 1009/2382
[INFO] processing batch 1010/2382
[INFO] processing batch 1011/2382
[INFO] processing batch 1012/2382
[INFO] processing batch 1013/2382
[INFO] processing batch 1014/2382
[INFO] processing batch 1015/2382
[INFO] processing batch 1016/2382
[INFO] processing batch 1017/2382
[INFO] processing batch 1018/2382
[INFO] processing batch 1019/2382
[INFO] processing batch 1020/2382
[INFO] processing batch 1021/2382
[INFO] processing batch 1022/2382
[INFO] processing batch 1023/2382
[INFO] processing batch 1024/2382
[INFO] processing batch 1025/2382
[INFO] processing batch 1026/2382
[INFO] processing batch 1027/2382
[INFO] processing batch 1028/2382
[INFO] process

[INFO] processing batch 1241/2382
[INFO] processing batch 1242/2382
[INFO] processing batch 1243/2382
[INFO] processing batch 1244/2382
[INFO] processing batch 1245/2382
[INFO] processing batch 1246/2382
[INFO] processing batch 1247/2382
[INFO] processing batch 1248/2382
[INFO] processing batch 1249/2382
[INFO] processing batch 1250/2382
[INFO] processing batch 1251/2382
[INFO] processing batch 1252/2382
[INFO] processing batch 1253/2382
[INFO] processing batch 1254/2382
[INFO] processing batch 1255/2382
[INFO] processing batch 1256/2382
[INFO] processing batch 1257/2382
[INFO] processing batch 1258/2382
[INFO] processing batch 1259/2382
[INFO] processing batch 1260/2382
[INFO] processing batch 1261/2382
[INFO] processing batch 1262/2382
[INFO] processing batch 1263/2382
[INFO] processing batch 1264/2382
[INFO] processing batch 1265/2382
[INFO] processing batch 1266/2382
[INFO] processing batch 1267/2382
[INFO] processing batch 1268/2382
[INFO] processing batch 1269/2382
[INFO] process

[INFO] processing batch 1482/2382
[INFO] processing batch 1483/2382
[INFO] processing batch 1484/2382
[INFO] processing batch 1485/2382
[INFO] processing batch 1486/2382
[INFO] processing batch 1487/2382
[INFO] processing batch 1488/2382
[INFO] processing batch 1489/2382
[INFO] processing batch 1490/2382
[INFO] processing batch 1491/2382
[INFO] processing batch 1492/2382
[INFO] processing batch 1493/2382
[INFO] processing batch 1494/2382
[INFO] processing batch 1495/2382
[INFO] processing batch 1496/2382
[INFO] processing batch 1497/2382
[INFO] processing batch 1498/2382
[INFO] processing batch 1499/2382
[INFO] processing batch 1500/2382
[INFO] processing batch 1501/2382
[INFO] processing batch 1502/2382
[INFO] processing batch 1503/2382
[INFO] processing batch 1504/2382
[INFO] processing batch 1505/2382
[INFO] processing batch 1506/2382
[INFO] processing batch 1507/2382
[INFO] processing batch 1508/2382
[INFO] processing batch 1509/2382
[INFO] processing batch 1510/2382
[INFO] process

[INFO] processing batch 1723/2382
[INFO] processing batch 1724/2382
[INFO] processing batch 1725/2382
[INFO] processing batch 1726/2382
[INFO] processing batch 1727/2382
[INFO] processing batch 1728/2382
[INFO] processing batch 1729/2382
[INFO] processing batch 1730/2382
[INFO] processing batch 1731/2382
[INFO] processing batch 1732/2382
[INFO] processing batch 1733/2382
[INFO] processing batch 1734/2382
[INFO] processing batch 1735/2382
[INFO] processing batch 1736/2382
[INFO] processing batch 1737/2382
[INFO] processing batch 1738/2382
[INFO] processing batch 1739/2382
[INFO] processing batch 1740/2382
[INFO] processing batch 1741/2382
[INFO] processing batch 1742/2382
[INFO] processing batch 1743/2382
[INFO] processing batch 1744/2382
[INFO] processing batch 1745/2382
[INFO] processing batch 1746/2382
[INFO] processing batch 1747/2382
[INFO] processing batch 1748/2382
[INFO] processing batch 1749/2382
[INFO] processing batch 1750/2382
[INFO] processing batch 1751/2382
[INFO] process

[INFO] processing batch 1964/2382
[INFO] processing batch 1965/2382
[INFO] processing batch 1966/2382
[INFO] processing batch 1967/2382
[INFO] processing batch 1968/2382
[INFO] processing batch 1969/2382
[INFO] processing batch 1970/2382
[INFO] processing batch 1971/2382
[INFO] processing batch 1972/2382
[INFO] processing batch 1973/2382
[INFO] processing batch 1974/2382
[INFO] processing batch 1975/2382
[INFO] processing batch 1976/2382
[INFO] processing batch 1977/2382
[INFO] processing batch 1978/2382
[INFO] processing batch 1979/2382
[INFO] processing batch 1980/2382
[INFO] processing batch 1981/2382
[INFO] processing batch 1982/2382
[INFO] processing batch 1983/2382
[INFO] processing batch 1984/2382
[INFO] processing batch 1985/2382
[INFO] processing batch 1986/2382
[INFO] processing batch 1987/2382
[INFO] processing batch 1988/2382
[INFO] processing batch 1989/2382
[INFO] processing batch 1990/2382
[INFO] processing batch 1991/2382
[INFO] processing batch 1992/2382
[INFO] process

[INFO] processing batch 2205/2382
[INFO] processing batch 2206/2382
[INFO] processing batch 2207/2382
[INFO] processing batch 2208/2382
[INFO] processing batch 2209/2382
[INFO] processing batch 2210/2382
[INFO] processing batch 2211/2382
[INFO] processing batch 2212/2382
[INFO] processing batch 2213/2382
[INFO] processing batch 2214/2382
[INFO] processing batch 2215/2382
[INFO] processing batch 2216/2382
[INFO] processing batch 2217/2382
[INFO] processing batch 2218/2382
[INFO] processing batch 2219/2382
[INFO] processing batch 2220/2382
[INFO] processing batch 2221/2382
[INFO] processing batch 2222/2382
[INFO] processing batch 2223/2382
[INFO] processing batch 2224/2382
[INFO] processing batch 2225/2382
[INFO] processing batch 2226/2382
[INFO] processing batch 2227/2382
[INFO] processing batch 2228/2382
[INFO] processing batch 2229/2382
[INFO] processing batch 2230/2382
[INFO] processing batch 2231/2382
[INFO] processing batch 2232/2382
[INFO] processing batch 2233/2382
[INFO] process

In [17]:
feature_extract(target_path, 64, labels_t)

[INFO] processing batch 1/866
[INFO] processing batch 2/866
[INFO] processing batch 3/866
[INFO] processing batch 4/866
[INFO] processing batch 5/866
[INFO] processing batch 6/866
[INFO] processing batch 7/866
[INFO] processing batch 8/866
[INFO] processing batch 9/866
[INFO] processing batch 10/866
[INFO] processing batch 11/866
[INFO] processing batch 12/866
[INFO] processing batch 13/866
[INFO] processing batch 14/866
[INFO] processing batch 15/866
[INFO] processing batch 16/866
[INFO] processing batch 17/866
[INFO] processing batch 18/866
[INFO] processing batch 19/866
[INFO] processing batch 20/866
[INFO] processing batch 21/866
[INFO] processing batch 22/866
[INFO] processing batch 23/866
[INFO] processing batch 24/866
[INFO] processing batch 25/866
[INFO] processing batch 26/866
[INFO] processing batch 27/866
[INFO] processing batch 28/866
[INFO] processing batch 29/866
[INFO] processing batch 30/866
[INFO] processing batch 31/866
[INFO] processing batch 32/866
[INFO] processing

[INFO] processing batch 261/866
[INFO] processing batch 262/866
[INFO] processing batch 263/866
[INFO] processing batch 264/866
[INFO] processing batch 265/866
[INFO] processing batch 266/866
[INFO] processing batch 267/866
[INFO] processing batch 268/866
[INFO] processing batch 269/866
[INFO] processing batch 270/866
[INFO] processing batch 271/866
[INFO] processing batch 272/866
[INFO] processing batch 273/866
[INFO] processing batch 274/866
[INFO] processing batch 275/866
[INFO] processing batch 276/866
[INFO] processing batch 277/866
[INFO] processing batch 278/866
[INFO] processing batch 279/866
[INFO] processing batch 280/866
[INFO] processing batch 281/866
[INFO] processing batch 282/866
[INFO] processing batch 283/866
[INFO] processing batch 284/866
[INFO] processing batch 285/866
[INFO] processing batch 286/866
[INFO] processing batch 287/866
[INFO] processing batch 288/866
[INFO] processing batch 289/866
[INFO] processing batch 290/866
[INFO] processing batch 291/866
[INFO] p

[INFO] processing batch 518/866
[INFO] processing batch 519/866
[INFO] processing batch 520/866
[INFO] processing batch 521/866
[INFO] processing batch 522/866
[INFO] processing batch 523/866
[INFO] processing batch 524/866
[INFO] processing batch 525/866
[INFO] processing batch 526/866
[INFO] processing batch 527/866
[INFO] processing batch 528/866
[INFO] processing batch 529/866
[INFO] processing batch 530/866
[INFO] processing batch 531/866
[INFO] processing batch 532/866
[INFO] processing batch 533/866
[INFO] processing batch 534/866
[INFO] processing batch 535/866
[INFO] processing batch 536/866
[INFO] processing batch 537/866
[INFO] processing batch 538/866
[INFO] processing batch 539/866
[INFO] processing batch 540/866
[INFO] processing batch 541/866
[INFO] processing batch 542/866
[INFO] processing batch 543/866
[INFO] processing batch 544/866
[INFO] processing batch 545/866
[INFO] processing batch 546/866
[INFO] processing batch 547/866
[INFO] processing batch 548/866
[INFO] p

[INFO] processing batch 775/866
[INFO] processing batch 776/866
[INFO] processing batch 777/866
[INFO] processing batch 778/866
[INFO] processing batch 779/866
[INFO] processing batch 780/866
[INFO] processing batch 781/866
[INFO] processing batch 782/866
[INFO] processing batch 783/866
[INFO] processing batch 784/866
[INFO] processing batch 785/866
[INFO] processing batch 786/866
[INFO] processing batch 787/866
[INFO] processing batch 788/866
[INFO] processing batch 789/866
[INFO] processing batch 790/866
[INFO] processing batch 791/866
[INFO] processing batch 792/866
[INFO] processing batch 793/866
[INFO] processing batch 794/866
[INFO] processing batch 795/866
[INFO] processing batch 796/866
[INFO] processing batch 797/866
[INFO] processing batch 798/866
[INFO] processing batch 799/866
[INFO] processing batch 800/866
[INFO] processing batch 801/866
[INFO] processing batch 802/866
[INFO] processing batch 803/866
[INFO] processing batch 804/866
[INFO] processing batch 805/866
[INFO] p

In [ ]:
def data_load(dirs, lab):
    
    x, y = [], []
    path = glob.glob(os.path.join(dirs, lab))
    print(path)
    for label, img in enumerate(path):
        print(label, img)
        for i in glob.glob(os.path.join(img, '*.*')):
            #print(i, label)
            x.append(scipy.ndimage.imread(i, mode='RGB'))
            y.append(label)
    
    return np.asarray(x), np.asarray(y)

In [ ]:
x_, y_ = [], []

#Run for source data
for i in dir_list1:
    for j in i:
    
        x, y = data_load(SOURCE_DIR, j)
        print('Finished reading source data for class', j)

        x_.extend(x)
        y_.extend(y)
    
    
xsource = np.asarray(x_)
ysource = np.asarray(y_)

In [ ]:
(xsource.shape, ysource.shape)

In [ ]:
h5f = h5py.File(os.path.join(DATA_LOAD_DIR, 'xsource2.h5'), 'w')
h5f.create_dataset('xsource_2', data = xsource)

In [ ]:
h5f_ = h5py.File(os.path.join(DATA_LOAD_DIR, 'ysource2.h5'), 'w')
h5f_.create_dataset('ysource_2', data = ysource)

In [ ]:
h5f.close()

In [ ]:
h5f_.close()

In [ ]:
x_t, y_t = [], []

#Run for target data
for i in dir_list1:
    for j in i:
        
        xt, yt = data_load(TARGET_DIR, j)
        print('Finished reading target data for class', j)
        x_t.extend(xt)
        y_t.extend(yt)
    
xtarget = np.asarray(x_t)
ytarget = np.asarray(y_t)

In [ ]:
(xtarget.shape, ytarget.shape)

In [ ]:
h5f = h5py.File(os.path.join(DATA_LOAD_DIR, 'xtarget1.h5'), 'w')
h5f.create_dataset('xtarget_1', data = xtarget)

In [ ]:
h5f_ = h5py.File(os.path.join(DATA_LOAD_DIR, 'ytarget1.h5'), 'w')
h5f_.create_dataset('ytarget_1', data = ytarget)

In [ ]:
h5f.close()

In [ ]:
h5f_.close()